# lib

In [37]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.logger import configure
import stable_baselines3 as sb3
import sb3_contrib  as sb3_trib
from sb3_contrib import TRPO
from stable_baselines3.common.callbacks import StopTrainingOnRewardThreshold, EvalCallback
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnNoModelImprovement
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from stable_baselines3.common.monitor import Monitor # Import the Monitor wrapper
import os
import time

import math

# preset

In [38]:
soc_preset = 0.1

In [39]:
Action_arr = pd.DataFrame(columns=["Ge","Re",'l1','REavai','Ebess','EbessMAX','EbessMIN','soc'])
df_each_step = pd.DataFrame(columns=["step", "action", "state", "total_reward", "lamda", "soc"])
df_each_episode = pd.DataFrame(columns=["Ge_ar", "Re_ar", "Be_ar", "soc", "load", "reward"])

load_standard = np.array([439,417,394,376,386,353,339,351,385,422,439,443,458,453,427,394,370,369,415,478,460,466,480,457], dtype=np.float32)
PVAvai_standard = np.array([0,0,0,0,0,13.2,48.9,104,177,251,306,313,315,279,240,156,64.3,37.4,0,0,0,0,0,0], dtype=np.float32)
WindAvai_standard =np.array([149,153,151,152,151,147,150,149,150,149,151,154,157,157,158,154,159,156,157,157,162,158,158,162])*2
ReAvai_standard = PVAvai_standard + WindAvai_standard


# env

In [40]:
def map_value(x, in_min, in_max, out_min, out_max):
    # Công thức chuyển đổi giá trị
    return (x - in_min) / (in_max - in_min) * (out_max - out_min) + out_min


## trainng env

In [41]:
class MinimizeProductEnv_24_4(gym.Env):
    def __init__(self):
        super(MinimizeProductEnv_24_4, self).__init__()
        # Các giá trị của hệ thống
        self.max_steps = 24
        self.load_standard = load_standard
        self.ReAvai_standard = ReAvai_standard
        self.noise_percentage = 0.1
        self.load = None
        self.ReAvai = None
        self.soc_prev = soc_preset
        self.cap_max = 2000.0
        self.P_bess_max = 550.0

        # Obs: soc
        self.observation_space = spaces.Box(
            low=np.array([-10, 0, 0, 0], dtype=np.float32),
            high=np.array([10, np.inf, np.inf, self.max_steps], dtype=np.float32)
        )

        # action  Gen, RE
        self.action_space =  spaces.Box(low=np.array([-1,-1], dtype=np.float32),
                                     high=np.array([1,1], dtype=np.float32))
        self.total_reward = 0
        self.Ge_ar = []
        self.Re_ar = []
        self.Be_ar = []
        self.soc = []
        self.load_ = []

    def _add_percentage_noise(self, data, percentage=0.1):
        lower_bound = 1 - percentage
        upper_bound = 1 + percentage
        factors = np.random.uniform(lower_bound, upper_bound, data.shape)
        noisy_data = data * factors
        return noisy_data

    def reset(self, seed=None, options=None, currStep=0, loadForecast=None, REforecast=None, pastRW=0, soc_preset=soc_preset):
        super().reset(seed=seed)
        self.step_count = currStep
        self.total_reward = pastRW
        # Tạo bộ dữ liệu load và ReAvai mới với nhiễu cho mỗi episode
        self.load = self._add_percentage_noise(self.load_standard, self.noise_percentage)
        self.ReAvai = self._add_percentage_noise(self.ReAvai_standard, self.noise_percentage)
        self.soc_prev = soc_preset
        self.Ge_ar = []
        self.Re_ar = []
        self.Be_ar = []
        self.soc = []
        self.load_ = []
        self.state = np.array([
            self.soc_prev,
            self.load_standard[currStep],
            self.ReAvai_standard[currStep],
            float(currStep)
        ], dtype=np.float32)
        return self.state, {}

    def step(self, action):
        terminated = False
        truncated = False
        reward = 0
        # Cập nhật trạng thái với action
        Egen, Ere = action

        _max = self.ReAvai[self.step_count] # Sử dụng giá trị ReAvai đã bị nhiễu cho bước hiện tại

        Ere=map_value(Ere,-1,1,0,_max)
        Ebess_max=min(self.P_bess_max,self.soc_prev*self.cap_max)
        Ebess_min=max(-self.P_bess_max,-(1-self.soc_prev)*self.cap_max)
        Egen= map_value(Egen,-1,1,0,2000) # Sử dụng giá trị load đã bị nhiễu cho bước hiện tại


        #tính Ebess
        Ebess=self.load[self.step_count]-Egen-Ere
        soc = self.soc_prev-Ebess/self.cap_max
        reward = -(
            Egen
            -(Ere-self.ReAvai[self.step_count])
            + 1e6 * ( max(0, Ebess - Ebess_max) + max(0, Ebess_min - Ebess) )
            )
        self.reward = reward
        self.soc_prev=soc

        self.total_reward += (reward)

        self.step_count += 1

        if self.step_count >= self.max_steps:
            self.state = np.array([
            soc,                                     # Updated SOC
            0,             # Current load (vẫn là giá trị bị nhiễu của episode này)
            0,           # Available renewable energy (vẫn là giá trị bị nhiễu của episode này)
            float(self.step_count)                  # Current time step
            ], dtype=np.float32)
        else:
            self.state = np.array([
              soc,                                     # Updated SOC
              self.load_standard[self.step_count],             # Current load (vẫn là giá trị bị nhiễu của episode này)
              self.ReAvai_standard[self.step_count],           # Available renewable energy (vẫn là giá trị bị nhiễu của episode này)
              float(self.step_count)                  # Current time step
              ], dtype=np.float32)


        # Điều kiện kết thúc episode
        # SOC vượt ngưỡng THẬT SỰ (trước khi clip)
        terminated = (self.step_count >= 24)
        # Hết bước mà không bị terminated sớm
        #truncated = (self.step_count >= 23)

        terminated = bool(terminated) # or terminated = True if terminated else False
        truncated = bool(truncated)

        return self.state, self.reward, terminated,truncated, {}



    # Phương thức render: Hiển thị trạng thái hiện tại của môi trường
    def render(self):
        # In thông báo reset nếu là bước 0
        if self.step_count == 0:
            print("Environment reset. Ready to start.")
        # In thông tin chi tiết cho các bước sau bước 0
        elif self.step_count > 0:
            # step_idx là index của bước VỪA KẾT THÚC (step_count-1)
            step_idx = self.step_count - 1
            print(f"*************************** Step: {step_idx}/{self.max_steps-1} **********************************")

            # Kiểm tra index có hợp lệ với các list dữ liệu đã ghi lại không
            if step_idx < len(self.Ge_ar):
                print(f"  Action: Ge={self.Ge_ar[step_idx]:.2f}, Re={self.Re_ar[step_idx]:.2f}, Ebess={self.Be_ar[step_idx]:.2f}")
                # Sửa tên thuộc tính ở đây: dùng self.load_ và self.soc
                # Lưu ý: self.ReAvai là array dữ liệu gốc, không phải lịch sử hành động, nên dùng self.ReAvai[step_idx] là đúng.
                print(f"  State: Load={self.load_[step_idx]:.2f}, RE_Avail={self.ReAvai[step_idx]:.2f}, SOC={self.soc[step_idx]*100:.2f}%")

                # Bạn có thể in thêm thông tin Ebess_max/min nếu muốn (cần lưu lại trong info hoặc biến riêng)
                # print(f"  Ebess Limits: [{info['Ebess_min_limit']:.2f}, {info['Ebess_max_limit']:.2f}]") # Cần lưu info để in

                # self.reward là reward của bước hiện tại (vừa tính trong step),
                # self.total_reward là tổng reward tích lũy.
                # Khi render sau step, bạn in reward của bước vừa xong (step_idx).
                # Lưu ý: self.reward được cập nhật ở cuối step, nên khi render sau step_idx, nó đang giữ giá trị reward của step_idx.
                print(f"  Reward: {self.reward:.4f}, Total Reward (so far): {self.total_reward:.4f}")
            else:
                # Trường hợp này ít xảy ra nếu render được gọi đúng sau step
                print(f"  Render called at step {self.step_count} but no data logged for step {step_idx}.")


    # Phương thức close: Dọn dẹp tài nguyên (nếu cần)
    def close(self):
        # Đóng các kết nối, file, v.v. nếu có.
        pass # Hiện tại không có tài nguyên cần dọn dẹp đặc biệt

## Eval env

In [42]:
def _add_percentage_noise(data, percentage=0.1, seed=None):
    if seed is not None:
        np.random.seed(seed) # Đặt seed

    lower_bound = 1 - percentage
    upper_bound = 1 + percentage
    factors = np.random.rand(*data.shape) * (upper_bound - lower_bound) + lower_bound
    noisy_data = data * factors

    if seed is not None:
        np.random.seed(None) # Đặt lại trạng thái ngẫu nhiên (tùy chọn, nhưng cẩn thận)

    return noisy_data

class MinimizeProductEnv_eval_24_4(gym.Env):
    """
    Môi trường quản lý năng lượng với dữ liệu load/RE nhiễu.
    Khi is_eval_env=True, nó sử dụng bộ dữ liệu cố định cho đánh giá.
    """
    # Thêm is_eval_env và n_eval_episodes vào constructor
    def __init__(self, is_eval_env=False, n_eval_episodes=5, noise_percentage=0.1, max_steps=24):
        super(MinimizeProductEnv_eval_24_4, self).__init__()

        # Cấu hình môi trường
        self.max_steps = max_steps
        self.load_standard = load_standard.copy() # Copy để tránh sửa dữ liệu global
        self.ReAvai_standard = ReAvai_standard.copy()
        self.noise_percentage = noise_percentage
        self.cap_max = 2000.0
        self.P_bess_max = 550.0

        # Obs: soc
        self.observation_space = spaces.Box(
            low=np.array([-10, 0, 0, 0], dtype=np.float32),
            high=np.array([10, np.inf, np.inf, self.max_steps], dtype=np.float32)
        )
        # action  Gen, RE, lamda1, lamda2
        self.action_space =  spaces.Box(low=np.array([-1,-1], dtype=np.float32),
                                     high=np.array([1,1], dtype=np.float32))

        self.total_reward = 0
        self.Ge_ar = []
        self.Re_ar = []
        self.Be_ar = []
        self.soc = []
        self.load_ = []

        # Cấu hình cho chế độ đánh giá
        self.is_eval_env = is_eval_env
        self.n_eval_episodes = n_eval_episodes
        self._current_eval_episode = 0 # Theo dõi episode hiện tại trong chu kỳ đánh giá cố định
        self._fixed_eval_data = None # Sẽ lưu dữ liệu cố định nếu là env eval

        # Nếu instance này là môi trường cho đánh giá, tạo sẵn bộ dữ liệu cố định
        if self.is_eval_env:
            print(f"Initializing Evaluation Environment: Generating {n_eval_episodes} fixed noisy data cycles.")
            self._fixed_eval_data = []
            # Sử dụng một seed cố định (ví dụ: 42) để đảm bảo bộ 5 dữ liệu này
            # giống hệt nhau MỖI KHI script chạy instance này.
            initial_eval_seed = 60

            for i in range(self.n_eval_episodes):
                # Tạo bộ dữ liệu nhiễu cho mỗi episode trong N_EVAL_EPISODES đánh giá
                # Sử dụng seed+i để các bộ dữ liệu là khác nhau nhưng vẫn cố định qua các lần chạy
                noisy_load = _add_percentage_noise(self.load_standard, self.noise_percentage, seed=initial_eval_seed + i)
                noisy_reavai = _add_percentage_noise(self.ReAvai_standard, self.noise_percentage, seed=initial_eval_seed + i + 1000) # Seed khác cho ReAvai

                # Lưu bản copy của dữ liệu nhiễu đã tạo
                self._fixed_eval_data.append((noisy_load.copy(), noisy_reavai.copy()))
            print("Fixed evaluation data generated.")

    # Phương thức reset được sửa đổi để chọn nguồn dữ liệu
    def reset(self, *, seed=None, options=None, currStep=0, loadForecast=None, REforecast=None, pastRW=0, soc_preset=soc_preset):
        # Gọi reset của lớp cha để xử lý seed cho bộ ngẫu nhiên của Gym
        super().reset(seed=seed)

        self.step_count = currStep

        # --- Logic chọn nguồn dữ liệu Load/ReAvai ---
        if self.is_eval_env and self._fixed_eval_data is not None:
            # Nếu đây là môi trường đánh giá, lấy bộ dữ liệu cố định tương ứng
            # self._current_eval_episode sẽ vòng lặp qua 0, 1, 2, 3, 4, 0, 1, ...
            load_data_for_episode, re_data_for_episode = self._fixed_eval_data[self._current_eval_episode]
            self.load = load_data_for_episode.copy() # Gán dữ liệu đã lưu trữ
            self.ReAvai = re_data_for_episode.copy()

            # Chuẩn bị cho episode đánh giá tiếp theo trong chu kỳ cố định
            self._current_eval_episode = (self._current_eval_episode + 1) % self.n_eval_episodes
            # print(f"Eval Episode Index: {self._current_eval_episode}") # Có thể in để debug

        else:
             # Trường hợp này KHÔNG XẢY RA nếu instance được tạo với is_eval_env=True
             # nhưng giữ lại để đầy đủ (ví dụ dùng cho training env)
            print("Warning: Resetting evaluation env without fixed data.") # Chỉ in nếu có vấn đề
            self.load = _add_percentage_noise(self.load_standard, self.noise_percentage, seed=seed)
            self.ReAvai = _add_percentage_noise(self.ReAvai_standard, self.noise_percentage, seed=seed + 1 if seed is not None else None)

        # --- Khởi tạo các biến episode khác ---
        self.Ge_ar = []
        self.Re_ar = []
        self.Be_ar = []
        self.soc = []
        self.load_ = []
        self.reward = 0 # Reset reward bước hiện tại
        self.soc_prev = soc_preset
        self.total_reward = pastRW


        self.state = np.array([
            self.soc_prev,
            self.load_standard[currStep],
            self.ReAvai_standard[currStep],
            float(currStep)
        ], dtype=np.float32)

        # --- Có thể in dữ liệu đầu tiên của episode để kiểm tra ---
        # print(f"Episode Data (Load[0], RE[0]): {self.load[0]:.2f}, {self.ReAvai[0]:.2f}")
        # print(f"State after reset: {self.state}")

        return self.state, {}

    # Phương thức step: Thực hiện một bước trong môi trường
    def step(self, action):
        terminated = False
        truncated = False

        # Cập nhật trạng thái với action
        Egen, Ere = action

        _min = 0
        _max = self.ReAvai[self.step_count] # Sử dụng giá trị ReAvai đã bị nhiễu cho bước hiện tại

        Ere=map_value(Ere,-1,1,_min,_max)
        Ebess_max=min(self.P_bess_max,self.soc_prev*self.cap_max)
        Ebess_min=max(-self.P_bess_max,-(1-self.soc_prev)*self.cap_max)
        Egen= map_value(Egen,-1,1,0,2000) # Sử dụng giá trị load đã bị nhiễu cho bước hiện tại


        #tính Ebess
        Ebess=self.load[self.step_count]-Egen-Ere
        soc = self.soc_prev-Ebess/self.cap_max
        reward = -(
            Egen
            -(Ere-self.ReAvai[self.step_count])
            + 1e6 * ( max(0, Ebess - Ebess_max) + max(0, Ebess_min - Ebess) )
            )
        self.reward = reward
        self.soc_prev=soc

        self.total_reward += (reward)

        self.step_count += 1

        if self.step_count >= self.max_steps:
            self.state = np.array([
            soc,                                     # Updated SOC
            0,             # Current load (vẫn là giá trị bị nhiễu của episode này)
            0,           # Available renewable energy (vẫn là giá trị bị nhiễu của episode này)
            float(self.step_count)                  # Current time step
            ], dtype=np.float32)
        else:
            self.state = np.array([
              soc,                                     # Updated SOC
              self.load_standard[self.step_count],             # Current load (vẫn là giá trị bị nhiễu của episode này)
              self.ReAvai_standard[self.step_count],           # Available renewable energy (vẫn là giá trị bị nhiễu của episode này)
              float(self.step_count)                  # Current time step
              ], dtype=np.float32)


        # Điều kiện kết thúc episode
        # SOC vượt ngưỡng THẬT SỰ (trước khi clip)
        terminated = (soc > 1.0) or (soc < 0.0)
        # Hết bước mà không bị terminated sớm
        truncated = (self.step_count >= 24)

        terminated = bool(terminated) # or terminated = True if terminated else False
        truncated = bool(truncated)


        return self.state, self.reward, terminated, truncated, {}

    # Phương thức render: Hiển thị trạng thái hiện tại của môi trường
    def render(self):
        # In thông báo reset nếu là bước 0
        if self.step_count == 0:
            print("Environment reset. Ready to start.")
        # In thông tin chi tiết cho các bước sau bước 0
        elif self.step_count > 0:
            # step_idx là index của bước VỪA KẾT THÚC (step_count-1)
            step_idx = self.step_count - 1
            print(f"*************************** Step: {step_idx}/{self.max_steps-1} **********************************")

            # Kiểm tra index có hợp lệ với các list dữ liệu đã ghi lại không
            if step_idx < len(self.Ge_ar):
                print(f"  Action: Ge={self.Ge_ar[step_idx]:.2f}, Re={self.Re_ar[step_idx]:.2f}, Ebess={self.Be_ar[step_idx]:.2f}")
                print(f"  State: Load={self.load_[step_idx]:.2f}, RE_Avail={self.ReAvai[step_idx]:.2f}, SOC={self.soc[step_idx]*100:.2f}%")

                print(f"  Reward: {self.reward:.4f}, Total Reward (so far): {self.total_reward:.4f}")
            else:
                print(f"  Render called at step {self.step_count} but no data logged for step {step_idx}.")


    # Phương thức close: Dọn dẹp tài nguyên
    def close(self):
        pass

# runing

In [43]:
env = MinimizeProductEnv_24_4()
eval_env = MinimizeProductEnv_eval_24_4(is_eval_env=True, n_eval_episodes=5, noise_percentage=0.1, max_steps=24)
# real_env = MinimizeProductEnv_real_24_4()
check_env(env)
check_env(eval_env)
# check_env(real_env)

Initializing Evaluation Environment: Generating 5 fixed noisy data cycles.
Fixed evaluation data generated.


In [44]:
env.reset()
print(env.observation_space)
print(env.action_space)
print(env.soc_prev)

Box([-10.   0.   0.   0.], [10. inf inf 24.], (4,), float32)
Box(-1.0, 1.0, (2,), float32)
0.1


In [45]:
eval_env.reset()
print(eval_env.observation_space)
print(eval_env.action_space)
print(eval_env.soc_prev)

Box([-10.   0.   0.   0.], [10. inf inf 24.], (4,), float32)
Box(-1.0, 1.0, (2,), float32)
0.1


## training

In [46]:
train_env = env
eval_env = eval_env
eval_env = Monitor(eval_env)

# 3. Thiết lập đường dẫn lưu log và mô hình
log_dir = os.path.join( "training_logs_final_env_15_5")
eval_log_path = os.path.join(log_dir, "eval_logs")
best_model_save_path = os.path.join(log_dir, "best_model_15_5_soc_0.1") # EvalCallback sẽ lưu mô hình tốt nhất tại đây

os.makedirs(eval_log_path, exist_ok=True)
os.makedirs(best_model_save_path, exist_ok=True)

# 4. Tạo StopTrainingOnRewardThreshold
# Callback này sẽ theo dõi chỉ số "eval/mean_reward" được ghi bởi EvalCallback
callback_on_best = StopTrainingOnRewardThreshold(reward_threshold=0, verbose=1)


# 5. Tạo EvalCallback
# Đánh giá mỗi 240 bước thời gian huấn luyện
# Chạy 5 episode cho mỗi lần đánh giá
# Lưu log đánh giá vào eval_log_path
# Lưu mô hình tốt nhất vào best_model_save_path
eval_callback = EvalCallback(
    eval_env,
    callback_on_new_best=callback_on_best,
    best_model_save_path=best_model_save_path,
    log_path=eval_log_path,
    eval_freq=2064*2,
    n_eval_episodes=5,
    deterministic=True # Sử dụng chính sách tất định trong khi đánh giá
)


In [47]:
start_time = time.time()


In [48]:
model = sb3_trib.TRPO("MlpPolicy", train_env, tensorboard_log=log_dir, n_steps= 2064, batch_size = 120, gamma = 0.995, verbose = 0)
model.learn(total_timesteps=24*100000, callback=eval_callback, progress_bar=False)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Thời gian chạy: {elapsed_time} giây")

c:\Users\KJD1912\AppData\Local\Programs\Python\Python312\Lib\site-packages\sb3_contrib\trpo\trpo.py:154: UserWarning: You have specified a mini-batch size of 120, but because the `RolloutBuffer` is of size `n_steps * n_envs = 2064`, after every 17 untruncated mini-batches, there will be a truncated mini-batch of size 24
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2064 and n_envs=1)
  warnings.warn(


Eval num_timesteps=4128, episode_reward=-616553910.20 +/- 51983341.98
Episode length: 4.00 +/- 0.00
New best mean reward!
Eval num_timesteps=8256, episode_reward=-218036403.79 +/- 115707403.91
Episode length: 6.80 +/- 0.40
New best mean reward!
Eval num_timesteps=12384, episode_reward=-22608830.31 +/- 10381662.66
Episode length: 4.60 +/- 0.80
New best mean reward!
Eval num_timesteps=16512, episode_reward=-121504109.42 +/- 24321823.66
Episode length: 1.00 +/- 0.00
Eval num_timesteps=20640, episode_reward=-104497221.80 +/- 24398119.10
Episode length: 1.00 +/- 0.00
Eval num_timesteps=24768, episode_reward=-114355376.21 +/- 49477662.27
Episode length: 1.80 +/- 0.40
Eval num_timesteps=28896, episode_reward=-81930084.23 +/- 70659190.40
Episode length: 1.40 +/- 0.49
Eval num_timesteps=33024, episode_reward=-72200164.96 +/- 68916866.64
Episode length: 1.20 +/- 0.40
Eval num_timesteps=37152, episode_reward=-70571359.18 +/- 68430733.49
Episode length: 1.20 +/- 0.40
Eval num_timesteps=41280, epis

KeyboardInterrupt: 

In [51]:
%load_ext tensorboard
%tensorboard --logdir="training_logs_final_env_15_5"
%reload_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 6428), started 0:00:35 ago. (Use '!kill 6428' to kill it.)